In [0]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("YourAppName") \
    .getOrCreate()

In [0]:
spark.conf.set("spark.databricks.io.cache.enabled", "false")
print("spark.databricks.io.cache.enabled is %s" % spark.conf.get("spark.databricks.io.cache.enabled"))

spark.databricks.io.cache.enabled is false


In [0]:
%pip install vaex

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import pandas as pd
import numpy as np
import vaex

print('pandas version: %s' % pd.__version__)

print('numpy version: %s' % np.__version__)

print('vaex version: %s' % vaex.__version__)

import pyarrow
print('pyarrow version: %s' % pyarrow.__version__)

import pyspark
print('pyspark version: %s' % pyspark.__version__)


import time

def benchmark(f, df, benchmarks, name, **kwargs):
    """Benchmark the given function against the given DataFrame.
    
    Parameters
    ----------
    f: function to benchmark
    df: data frame
    benchmarks: container for benchmark results
    name: task name
    
    Returns
    -------
    Duration (in seconds) of the given operation
    """
    start_time = time.time()
    ret = f(df, **kwargs)
    benchmarks['duration'].append(time.time() - start_time)
    benchmarks['task'].append(name)
    print(f"{name} took: {benchmarks['duration'][-1]} seconds")
    return benchmarks['duration'][-1]

def get_results(benchmarks):
    """Return a pandas DataFrame containing benchmark results."""
    return pd.DataFrame.from_dict(benchmarks)

pandas version: 2.2.2
numpy version: 1.26.4
vaex version: {'vaex': '4.17.0', 'vaex-core': '4.17.1', 'vaex-viz': '0.5.4', 'vaex-hdf5': '0.14.1', 'vaex-server': '0.9.0', 'vaex-astro': '0.9.3', 'vaex-jupyter': '0.8.2', 'vaex-ml': '0.18.3'}
pyarrow version: 7.0.0
pyspark version: 3.3.2.dev0


In [0]:
import pandas as pd

file_path = "/FileStore/tables/yellow_tripdata_2023_01.parquet"
#pandas_data = pd.read_parquet(file_path, engine="pyarrow")
pandas_data = spark.read.format('parquet').options(header='true').load(file_path).toPandas()

In [0]:
vaex_data = vaex.from_pandas(pandas_data)

vaex_benchmarks = {
    'duration': [],  # in seconds
    'task': [],
}
vaex_data.head()

#,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10.000000000,2023-01-01 00:40:36.000000000,1,0.97,1,N,161,141,2,9.3,1,0.5,0,0,1,14.3,2.5,0
1,2,2023-01-01 00:55:08.000000000,2023-01-01 01:01:27.000000000,1,1.1,1,N,43,237,1,7.9,1,0.5,4,0,1,16.9,2.5,0
2,2,2023-01-01 00:25:04.000000000,2023-01-01 00:37:49.000000000,1,2.51,1,N,48,238,1,14.9,1,0.5,15,0,1,34.9,2.5,0
3,1,2023-01-01 00:03:48.000000000,2023-01-01 00:13:25.000000000,0,1.9,1,N,138,7,1,12.1,7.25,0.5,0,0,1,20.85,0,1.25
4,2,2023-01-01 00:10:29.000000000,2023-01-01 00:21:19.000000000,1,1.43,1,N,107,79,1,11.4,1,0.5,3.28,0,1,19.68,2.5,0
5,2,2023-01-01 00:50:34.000000000,2023-01-01 01:02:52.000000000,1,1.84,1,N,161,137,1,12.8,1,0.5,10,0,1,27.8,2.5,0
6,2,2023-01-01 00:09:22.000000000,2023-01-01 00:19:49.000000000,1,1.66,1,N,239,143,1,12.1,1,0.5,3.42,0,1,20.52,2.5,0
7,2,2023-01-01 00:27:12.000000000,2023-01-01 00:49:56.000000000,1,11.7,1,N,142,200,1,45.7,1,0.5,10.74,3,1,64.44,2.5,0
8,2,2023-01-01 00:21:44.000000000,2023-01-01 00:36:40.000000000,1,2.95,1,N,164,236,1,17.7,1,0.5,5.68,0,1,28.38,2.5,0
9,2,2023-01-01 00:39:42.000000000,2023-01-01 00:50:36.000000000,1,3.01,1,N,141,107,2,14.9,1,0.5,0,0,1,19.9,2.5,0


In [0]:
# Read file (assuming Parquet format)
def read_file_parquet():
    return vaex.open("/FileStore/tables/yellow_tripdata_2023_01.parquet")

# Count rows
def count(df):
    return len(df)

# Count index length
def count_index_length(df):
    return len(df)

# Mean
def mean(df):
    return df['fare_amount'].mean()

# Standard deviation
def standard_deviation(df):
    return df['fare_amount'].std()

# Mean of sum
def mean_of_sum(df):
    return (df['fare_amount'] + df['tip_amount']).mean()

# Sum columns
def sum_columns(df):
    return (df['fare_amount'] + df['tip_amount']).sum()

# Mean of product
def mean_of_product(df):
    return (df['fare_amount'] * df['tip_amount']).mean()

# Product columns
def product_columns(df):
    return (df['fare_amount'] * df['tip_amount']).sum()

# Value counts
def value_counts(df):
    return df['fare_amount'].value_counts()

# Mean of complicated arithmetic operation
def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df['start_lon']
    phi_1 = df['start_lat']
    theta_2 = df['end_lon']
    phi_2 = df['end_lat']
    temp = (vaex.utils.deg2rad(theta_2 - theta_1) / 2).sin() ** 2 + (vaex.utils.deg2rad(phi_2 - phi_1) / 2).sin() ** 2
    ret = (2 * vaex.utils.rad2deg(temp.sqrt() / (1 - temp).sqrt())).mean()
    return ret

# Complicated arithmetic operation
def complicated_arithmetic_operation(df):
    theta_1 = df['start_lon']
    phi_1 = df['start_lat']
    theta_2 = df['end_lon']
    phi_2 = df['end_lat']
    temp = (vaex.utils.deg2rad(theta_2 - theta_1) / 2).sin() ** 2 + (vaex.utils.deg2rad(phi_2 - phi_1) / 2).sin() ** 2
    ret = 2 * vaex.utils.rad2deg(vaex.utils.arctan2(temp.sqrt(), (1 - temp).sqrt()))
    return ret

# Groupby statistics
def groupby_statistics(df):
    return df.groupby(by=df['passenger_count']).agg({
        'fare_amount': ['mean', 'std'],
        'tip_amount': ['mean', 'std']
    })

other = groupby_statistics(vaex_data)

# Join count
def join_count(df, other):
    return len(df.join(other, on='passenger_count', rsuffix='_other'))

# Join data
def join_data(df, other):
    return df.join(other, on='passenger_count', rsuffix='_other')

In [0]:
#benchmark(read_file_parquet, df=None, benchmarks=dask_benchmarks, name='read file')
benchmark(count, df=vaex_data, benchmarks=vaex_benchmarks, name='count')
benchmark(count_index_length, df=vaex_data, benchmarks=vaex_benchmarks, name='count index length')
benchmark(mean, df=vaex_data, benchmarks=vaex_benchmarks, name='mean')
benchmark(standard_deviation, df=vaex_data, benchmarks=vaex_benchmarks, name='standard deviation')
benchmark(mean_of_sum, df=vaex_data, benchmarks=vaex_benchmarks, name='mean of columns addition')
benchmark(sum_columns, df=vaex_data, benchmarks=vaex_benchmarks, name='addition of columns')
benchmark(mean_of_product, df=vaex_data, benchmarks=vaex_benchmarks, name='mean of columns multiplication')
benchmark(product_columns, df=vaex_data, benchmarks=vaex_benchmarks, name='multiplication of columns')
benchmark(value_counts, df=vaex_data, benchmarks=vaex_benchmarks, name='value counts')
# No column for this
# benchmark(mean_of_complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='mean of complex arithmetic ops')
# benchmark(complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='complex arithmetic ops')
benchmark(groupby_statistics, df=vaex_data, benchmarks=vaex_benchmarks, name='groupby statistics')
benchmark(join_count, vaex_data, benchmarks=vaex_benchmarks, name='join count', other=other)
benchmark(join_data, vaex_data, benchmarks=vaex_benchmarks, name='join', other=other) # cant join

count took: 1.4543533325195312e-05 seconds
count index length took: 6.4373016357421875e-06 seconds
mean took: 0.023628711700439453 seconds
standard deviation took: 0.13939714431762695 seconds
mean of columns addition took: 0.036054372787475586 seconds
addition of columns took: 0.03216671943664551 seconds
mean of columns multiplication took: 0.04404187202453613 seconds
multiplication of columns took: 0.030757427215576172 seconds
value counts took: 0.07884335517883301 seconds
groupby statistics took: 0.4891469478607178 seconds
join count took: 0.06205630302429199 seconds
join took: 0.05375051498413086 seconds
Out[5]: 0.05375051498413086

Operations with filtering

In [0]:
expr_filter = (vaex_data.tip_amount >= 1) & (vaex_data.tip_amount <= 5)

def filter_data(df):
    return df[expr_filter]
  
vaex_filtered = filter_data(vaex_data)

In [0]:
benchmark(count, vaex_filtered, benchmarks=vaex_benchmarks, name='filtered count')
benchmark(count_index_length, vaex_filtered, benchmarks=vaex_benchmarks, name='filtered count index length')
benchmark(mean, vaex_filtered, benchmarks=vaex_benchmarks, name='filtered mean')
benchmark(standard_deviation, vaex_filtered, benchmarks=vaex_benchmarks, name='filtered standard deviation')
benchmark(mean_of_sum, vaex_filtered, benchmarks=vaex_benchmarks, name ='filtered mean of columns addition')
benchmark(sum_columns, df=vaex_filtered, benchmarks=vaex_benchmarks, name='filtered addition of columns')
benchmark(mean_of_product, vaex_filtered, benchmarks=vaex_benchmarks, name ='filtered mean of columns multiplication')
benchmark(product_columns, df=vaex_filtered, benchmarks=vaex_benchmarks, name='filtered multiplication of columns')
#benchmark(mean_of_complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered mean of complex arithmetic ops')
#benchmark(complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered complex arithmetic ops')
benchmark(value_counts, vaex_filtered, benchmarks=vaex_benchmarks, name ='filtered value counts')
benchmark(groupby_statistics, vaex_filtered, benchmarks=vaex_benchmarks, name='filtered groupby statistics')

other = groupby_statistics(vaex_filtered)
# other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

benchmark(join_count, vaex_filtered, benchmarks=vaex_benchmarks, name='filtered join count', other=other)
benchmark(join_data, vaex_filtered, benchmarks=vaex_benchmarks, name='filtered join', other=other)

filtered count took: 0.018093347549438477 seconds
filtered count index length took: 1.0013580322265625e-05 seconds
filtered mean took: 0.04276633262634277 seconds
filtered standard deviation took: 0.11040163040161133 seconds
filtered mean of columns addition took: 0.06661796569824219 seconds
filtered addition of columns took: 0.06420516967773438 seconds
filtered mean of columns multiplication took: 0.06282472610473633 seconds
filtered multiplication of columns took: 0.05987715721130371 seconds
filtered value counts took: 0.09107613563537598 seconds
filtered groupby statistics took: 0.35552000999450684 seconds
filtered join count took: 0.05442380905151367 seconds
filtered join took: 0.05325722694396973 seconds
Out[7]: 0.05325722694396973

In [0]:
vaex_res_temp = get_results(vaex_benchmarks).set_index('task')
vaex_res_temp

,duration
task,
count,0.000015
count index length,0.000006
mean,0.023629
standard deviation,0.139397
mean of columns addition,0.036054
addition of columns,0.032167
mean of columns multiplication,0.044042
multiplication of columns,0.030757
value counts,0.078843
